In [ ]:
from sklearn.ensemble import GradientBoostingClassifier 
from kfold_and_metrics import *
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
df = pd.read_csv("final.csv")
df = df.drop(columns=["id"])
df.head()

In [ ]:
best_auc_average = 0
best = {}

for n_est in range(30, 201, 10):
    for m_depth in range(5, 31):
        for m_l_samples in range(5, 21):
            for crit in ['friedman_mse', 'squared_error']:
                params = {'n_estimators': n_est, 'max_depth': m_depth, 'min_samples_leaf': m_l_samples, 'criterion': crit}
                print("Current parameter combination:")
                for parameter, value in params.items():
                    print(f"\t{parameter}: {value}")
                print()

                model = GradientBoostingClassifier(n_estimators=n_est, max_depth=m_depth, min_samples_leaf=m_l_samples, criterion=crit)
                auc_results = k_fold_cv(model, df, metric_funcs=[roc_auc_score], pca_components=50, k_fold_verbose=True)
                auc_average, auc_std = weighted_avg_and_std(np.array(auc_results['roc_auc_score']))
                if auc_average > best_auc_average:
                    best_auc_average = auc_average
                    best = params

print("Results of the grid search parameter hypertunning:")
for parameter, value in best.items():
    print(f"\t{parameter}: {value}")